# Redes Neuronales Recurrentes

A lo largo de este notebook de prácticas haremos varios ejemplos, tanto con RNNs básicas como con LSTMs, y veremos las diferencias.

También trabajaremos con problemas con una única variable y con problemas multivariantes.

# 2. Predicción del valor de acciones en bolsa

A continuación vamos a ver un ejemplo más real y complejo, que además nos va a servir para introducir los problemas multivariantes.

Nos vamos a conectar a la API de Yahoo Stocks para descargarnos los últimos 20 años de datos de la cotización en bolsa de Amazon e intentar predecir valores futuros.

In [1]:
!pip install pandas-datareader

In [47]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [46]:
session.close()

In [2]:
import datetime as dt
from pandas_datareader import data

dataset_raw = data.DataReader('AMZN','yahoo', dt.datetime(2000,1,1), dt.datetime.now())
dataset_raw.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,89.5625,79.046875,81.5000,89.3750,16117600,89.3750
2000-01-04,91.5000,81.750000,85.3750,81.9375,17487400,81.9375
2000-01-05,75.1250,68.000000,70.5000,69.7500,38457400,69.7500
2000-01-06,72.6875,64.000000,71.3125,65.5625,18752000,65.5625
2000-01-07,70.5000,66.187500,67.0000,69.5625,10505400,69.5625


### **Ejercicio 4**

Trata de predecir el valor de cierre del precio de la acción de Amazon con la mayor exactitud posible. Para ello, puedes emplear tanto RNN como LSTM, elegir el número de celdas, el número de neuronas por celda, y el tamaño de ventana que consideres oportuno. También puedes elegir si quieres rellenar con 0's o no.

**NOTA**: Si decides emplear varias celdas, tienes que devolver las "secuencias" en las celdas que conectan con subsiguientes celdas. Aquí tienes un ejemplo:

```
model = Sequential()
model.add(SimpleRNN(64, input_shape=(look_back, variables), return_sequences=True)) # celda 1, 64 neuronas
model.add(SimpleRNN(32, return_sequences=True)) # celda 2, 32 neuronas
model.add(SimpleRNN(16)) # celda 3, 16 neuronas
model.add(Dense(8,activation='tanh')) # capa densa con 8 neuronas y activación tanh
model.add(Dense(1,activation='linear')) # capa de salida con 1 neurona y activación lineal
```

Para quedarte con la variable correspondiente al valor de cierre de la acción, puedes hacer lo siguiente:

```
dataframe = dataset_raw[['Close']]
```

**Es importante tener en cuenta que no por tener más capas va a funcionar mejor, ya que muy posiblemente ocasionemos overfitting.**

## Predicción con más de una variable

En esta ocasión utilizaremos todos los datos que nos brinda la API de Yahoo para predecir el precio de cierre de una acción.

In [3]:
import numpy as np
import datetime as dt
from pandas_datareader import data

dataset_raw = data.DataReader('AMZN','yahoo', dt.datetime(2000,1,1), dt.datetime.now())
dataset_raw.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,89.5625,79.046875,81.5000,89.3750,16117600,89.3750
2000-01-04,91.5000,81.750000,85.3750,81.9375,17487400,81.9375
2000-01-05,75.1250,68.000000,70.5000,69.7500,38457400,69.7500
2000-01-06,72.6875,64.000000,71.3125,65.5625,18752000,65.5625
2000-01-07,70.5000,66.187500,67.0000,69.5625,10505400,69.5625


In [4]:
# ligeramente modificada para escoger la variable objetivo
def create_dataset(dataset, look_back_memory=1, idx_target=-1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back_memory-1):
        dataX.append(dataset[i:i+look_back_memory])
        dataY.append(dataset[i+look_back_memory, idx_target])
    return np.array(dataX), np.array(dataY)

Si quisieramos utilizar solo 1 timestamp y nuestra variable objetivo fuese 'Close' (idx=3):

In [5]:
X, Y = create_dataset(dataset_raw.values, look_back_memory=1, idx_target=3)
print(X.shape)
print(Y.shape)

(5340, 1, 6)
(5340,)


Pero como hemos visto, funciona mejor con una ventana más amplia. Hagámoslo con 10 timestamps y así podremos compararlo luego con el caso de una sola variable y 10 timestamps que acabamos de hacer:

In [6]:
X, Y = create_dataset(dataset_raw.values, look_back_memory=10, idx_target=3)
print(X.shape)
print(Y.shape)

(5331, 10, 6)
(5331,)


In [10]:
# hacemos el import de todo lo que utilizaremos
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# fijamos la semilla para obtener resultados reproducibles
np.random.seed(42)

In [9]:
dataset_raw.shape

(5342, 6)

In [8]:
dataset_raw.describe()

,High,Low,Open,Close,Volume,Adj Close
count,5342.000000,5342.000000,5342.000000,5342.000000,5.342000e+03,5342.000000
mean,507.147677,495.363983,501.590964,501.453571,6.442460e+06,501.453571
std,775.802397,757.562098,767.425385,766.776660,5.093051e+06,766.776660
min,6.100000,5.510000,5.910000,5.970000,8.813000e+05,5.970000
25%,42.799999,41.145000,41.852499,41.992501,3.604900e+06,41.992501
50%,142.214996,138.825005,140.364998,140.110001,5.320400e+06,140.110001
75%,578.862503,562.124985,570.570007,571.517517,7.688900e+06,571.517517
max,3552.250000,3486.689941,3547.000000,3531.449951,1.043292e+08,3531.449951


In [11]:
dataframe = dataset_raw.drop(columns=['Adj Close'])
dataframe.describe()

,High,Low,Open,Close,Volume
count,5342.000000,5342.000000,5342.000000,5342.000000,5.342000e+03
mean,507.147677,495.363983,501.590964,501.453571,6.442460e+06
std,775.802397,757.562098,767.425385,766.776660,5.093051e+06
min,6.100000,5.510000,5.910000,5.970000,8.813000e+05
25%,42.799999,41.145000,41.852499,41.992501,3.604900e+06
50%,142.214996,138.825005,140.364998,140.110001,5.320400e+06
75%,578.862503,562.124985,570.570007,571.517517,7.688900e+06
max,3552.250000,3486.689941,3547.000000,3531.449951,1.043292e+08


In [12]:
# normalizamos el dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataframe)

In [13]:
dataset.min(axis=0)

array([0., 0., 0., 0., 0.])

In [14]:
dataset.max(axis=0)

array([1., 1., 1., 1., 1.])

A la hora de hacer las predicciones, des-normalizaremos el valor de Close utilizando su mínimo y su máximo (el penúltimo valor de los siguientes vectores):

In [15]:
scaler.data_min_

array([6.09999990e+00, 5.51000023e+00, 5.90999985e+00, 5.96999979e+00,
       8.81300000e+05])

In [16]:
scaler.data_max_

array([3.55225000e+03, 3.48668994e+03, 3.54700000e+03, 3.53144995e+03,
       1.04329200e+08])

In [17]:
# dividimos en train y test
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [18]:
# transformamos los datos para crearnos N registros con T timestamps cada uno 
# (uno por cada instante temporal hasta completar el tamaño de la ventana) y 
# las V variables de las que disponga nuestro dataset. En este caso, vamos a 
# escoger una ventana con un único timestamp T=1 y solo tendremos una variable,
# con lo que V=10 (número de pasajeros).
look_back_memory = 10
trainX, trainY = create_dataset(train, look_back_memory)
testX, testY = create_dataset(test, look_back_memory)
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(3568, 10, 5) (3568,)
(1752, 10, 5) (1752,)


In [19]:
# Nos aseguramos de que las dimensiones de las entradas son las correctas:
# (número de ventanas de T elementos, los T elementos de cada ventana, las V variables de cada timestamp)
variables = 5
trainX = np.reshape(trainX, (trainX.shape[0], look_back_memory, variables))
testX = np.reshape(testX, (testX.shape[0], look_back_memory, variables))
print(trainX.shape)
print(testX.shape)

(3568, 10, 5)
(1752, 10, 5)


In [48]:
# creamos el modelo y lo entrenamos
model = Sequential() #initialize model
model.add(LSTM(10, input_shape=(look_back_memory, variables)))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=1)

Epoch 1/20


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_3/lstm_3/PartitionedCall]] [Op:__inference_train_function_9709]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
# vamos a ver qué tal funciona nuestro modelo
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# una vez hechas las predicciones, tenemos que des-normalizarlas
xmin = scaler.data_min_[-2]
xmax = scaler.data_max_[-2]
trainPredict = trainPredict * (xmax - xmin) + xmin
trainY_orig = trainY * (xmax - xmin) + xmin
testPredict = testPredict * (xmax - xmin) + xmin
testY_orig = testY * (xmax - xmin) + xmin

In [ ]:
# y ahora calculamos el error cometido en train y en test
trainScore = math.sqrt(mean_squared_error(trainY_orig, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY_orig, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# por como creamos el dataset de entrenamiento, ahora tenemos que desplazar
# nuestras predicciones para que "cuadren" con el eje x de los datos originales
trainPredictPlot = np.full(dataset.shape[0], np.nan)
trainPredictPlot[look_back_memory:len(trainPredict)+look_back_memory] = trainPredict[:, 0]

# y lo mismo para el test
testPredictPlot = np.full(dataset.shape[0], np.nan)
testPredictPlot[len(trainPredict)+(look_back_memory*2)+1:len(dataset)-1] = testPredict[:, 0]

# y mostramos los datos originales, la predicción en training y la predicción en test
plt.plot(dataset_raw['Close'].values)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

**Más ejemplos interesantes de predicción con LSTMs**

- Predicción de potencia consumida en un hogar: 

https://www.analyticsvidhya.com/blog/2020/10/multivariate-multi-step-time-series-forecasting-using-stacked-lstm-sequence-to-sequence-autoencoder-in-tensorflow-2-0-keras/

https://databricks.com/blog/2019/09/10/doing-multivariate-time-series-forecasting-with-recurrent-neural-networks.html

- Predicción del precio de BitCoin: 

https://medium.com/@pierre.beaujuge/multivariate-time-series-forecasting-with-a-bidirectional-lstm-building-a-model-geared-to-4f020a160636

-  Predicción de la demanda de bicicletas:

https://curiousily.com/posts/demand-prediction-with-lstms-using-tensorflow-2-and-keras-in-python/


**Más datasets** 

https://archive.ics.uci.edu/ml/index.php

